In [1]:
# Start from md5 in Threats data, then search into OTX Alien Vault and 'use tags to look into CVEs of Nist'


In [2]:
import datetime
import os
from OTXv2 import OTXv2
import time
import json 
import pandas as pd 
import requests
import urllib3
import json
import sys

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

otx = OTXv2('276e51c0ef0cf3968203b219ac8bfe0a5bf68add0061c77ec1c118e7aac4e1fe')

In [3]:
mylist = []

for chunk in  pd.read_csv(r'C:\Users\leona\OneDrive\Desktop\ThreatsData\cleaned\threats.csv', sep=',', chunksize=20000):
    mylist.append(chunk)

threats_df = pd.concat(mylist, axis= 0)
del mylist

In [4]:
# Lookup of urls in OTX Alien Vault

In [5]:
md5_df = pd.DataFrame(threats_df[threats_df['ioc_type'] == 'md5_hash'][['ioc','ioc_type','file_name','signature','source','first_seen', 'last_seen','origin_country','tags']],).reset_index(drop = True)
md5_df

,ioc,ioc_type,file_name,signature,source,first_seen,last_seen,origin_country,tags
0,de9a87a8dc9eb67b3e54c452f63b2579,md5_hash,unknown,Unknown malware,Threat Fox,2023-04-25 08:19:19 UTC,NaN,NaN,"['APT', 'gamaredon']"
1,b0d72c957982061fcf88798428136039,md5_hash,unknown,Unknown malware,Threat Fox,2023-04-25 08:19:19 UTC,NaN,NaN,"['APT', 'gamaredon']"
2,975da522079b40365ef3801e71714eb4,md5_hash,win.zstealer,ZStealer,Threat Fox,2023-04-25 08:18:14 UTC,NaN,NaN,['ZStealer']
3,ab1448e5606aeebd04b298b41292bff2,md5_hash,win.zstealer,ZStealer,Threat Fox,2023-04-25 08:18:14 UTC,NaN,NaN,['ZStealer']
4,171755fb48a0ad6464e8808dc90856a9,md5_hash,win.zstealer,ZStealer,Threat Fox,2023-04-25 08:18:14 UTC,NaN,NaN,['ZStealer']
5,5b32164135c2798630f86fb25e787dde,md5_hash,win.zstealer,ZStealer,Threat Fox,2023-04-25 08:18:14 UTC,NaN,NaN,['ZStealer']
6,546970ad7aaaf199a426c8ab2c45a900,md5_hash,win.xworm,XWorm,Threat Fox,2023-04-21 18:01:47 UTC,NaN,NaN,['XWorm']
7,9742cd02a3d03c4e3394cf4d199ac8d6,md5_hash,win.ducktail,DUCKTAIL,Threat Fox,2023-04-21 18:01:23 UTC,NaN,NaN,"['APT', 'DUCKTAIL']"
8,8dc27ba3775ecc1d2b98b8a6f0d5ce7b,md5_hash,win.ducktail,DUCKTAIL,Threat Fox,2023-04-21 18:00:58 UTC,NaN,NaN,['DUCKTAIL']
9,aa6c29da0f89131f38fc6a0ea20abff0,md5_hash,win.ducktail,DUCKTAIL,Threat Fox,2023-04-21 18:00:31 UTC,NaN,NaN,"['APT', 'DUCKTAIL']"


In [6]:
otx_usefulColumns = ['otx_id','otx_name','otx_description', 'otx_tags', 'otx_attack_ids', 'otx_malware_families', 'otx_adversary', 'otx_modified', 'otx_created', 'otx_references', 'otx_author.username' ]

In [8]:
md5 = 'de9a87a8dc9eb67b3e54c452f63b2579'

current_pulse = otx.get('https://otx.alienvault.com/api/v1/indicators/file/'+ md5 +'/general')

tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse))

otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(tmp['pulse_info.pulses'])).T

otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(otxAlienVault_df[0])).add_prefix('otx_')
otxAlienVault_df

,otx_id,otx_name,otx_description,otx_modified,otx_created,otx_tags,otx_references,otx_public,otx_adversary,otx_targeted_countries,otx_malware_families,otx_attack_ids,otx_industries,otx_TLP,otx_cloned_from,otx_export_count,otx_upvotes_count,otx_downvotes_count,otx_votes_count,otx_locked,otx_pulse_source,otx_validator_count,otx_comment_count,otx_follower_count,otx_vote,otx_indicator_count,otx_is_author,otx_is_subscribing,otx_subscriber_count,otx_modified_text,otx_is_modified,otx_groups,otx_in_group,otx_threat_hunter_scannable,otx_threat_hunter_has_agents,otx_related_indicator_type,otx_related_indicator_is_active,otx_author.username,otx_author.id,otx_author.avatar_url,otx_author.is_subscribed,otx_author.is_following,otx_indicator_type_counts.URL,otx_indicator_type_counts.FileHash-MD5,otx_indicator_type_counts.IPv4
0,64485fdff729c635306141be,Twitter Feed - StopMalvertisin - 25-04-2023,,2023-04-25T23:18:55.147000,2023-04-25T23:18:55.147000,[],"[https://twitter.com/StopMalvertisin/status/1650745109519175680, https://twitter.com/StopMalvertisin/status/1650772102285500422, https://twitter.com/StopMalvertisin/status/1650923515955249154]",1,,[],[],[],[],green,None,3,0,0,0,False,api,0,0,0,0,11,False,None,270,14 days ago,False,[],False,True,1,FileHash-MD5,1,CyberHunterAutoFeed,182496,/otxapi/users/avatar_image/media/avatars/user_182496/resized/80/avatar_3b9c358f36.png,False,False,4,6,1


In [9]:
def getIPEntriesFromOTXAlienVault(df):
    
    final_df = pd.DataFrame()
        
    for i in range(len(df)):
        
        print(i)
        
        ip = df.at[i,'ioc'].split(':')[0]
        current_pulse = otx.get('https://otx.alienvault.com/api/v1/indicators/IPv4/'+ ip +'/general')
        tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse)).reset_index(drop = True)        
        otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(tmp['pulse_info.pulses'])).T
        
        if(otxAlienVault_df.empty == False):
        
            otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(otxAlienVault_df[0])).add_prefix('otx_')

            rows = otxAlienVault_df[otx_file_usefulColumns]

            tmp_row_df = pd.DataFrame()

            for j in range(len(rows)):

                row = df[df.index == i].join(pd.DataFrame(rows[rows.index == j]))
                tmp_row_df = pd.concat([tmp_row_df,row], ignore_index=True)

            final_df = pd.concat([final_df,tmp_row_df], ignore_index=True)
            
    return final_df

In [11]:
OTXAlienVault_url_df = getIPEntriesFromOTXAlienVault(md5_df)
OTXAlienVault_url_df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


""
